In [ ]:
%%writefile install_mfa.sh
!/bin/bash
## a script to install Montreal Forced Aligner (MFA)
root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# download mfa aligner
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge montreal-forced-aligner -y
# source $root_dir/miniconda3/bin/conda activate aligner

echo -e "\n======== DONE =========="


In [ ]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory
!bash ./install_mfa.sh {INSTALL_DIR}

In [ ]:
!cd /content/drive/MyDrive/test2
# FINALLY, align phonemes and speech
#"""MFA align requires "corpus_directory',"dictionary_path", "acoustic_model_path","output_directory""""
!source {INSTALL_DIR}/miniconda3/bin/activate aligner;\
mfa align --clean ./input english english ./output
# output files are at ./output file
!echo "See output files at ./output_file"